In [ ]:
import xlrd
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn import metrics

from keras.models import Sequential  
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling1D, AveragePooling1D
from keras.layers.convolutional import Conv1D  
from keras.utils import np_utils

import pywt

# Set global constants
_SCALE_FLAG_ = 1 


#Read datas and labels
ExcelFile = xlrd.open_workbook('data/Dataset_OV.xlsx')


# Read training data
Sheet_1 = ExcelFile.sheet_by_index(0)
train_x_list = list()
for i in range(Sheet_1.ncols-1):
    tpcol = Sheet_1.col_values(i+1, 1)
    train_x_list.append(tpcol)

train_x = np.array(train_x_list)
print(train_x.shape)

# Read training label
Sheet_1 = ExcelFile.sheet_by_index(2)
train_y = np.array(Sheet_1.col_values(0,1))
print(len(train_y))

# Read test data
Sheet_1 = ExcelFile.sheet_by_index(1)
test_x_list = list()
for i in range(Sheet_1.ncols-1):
    tpcol = Sheet_1.col_values(i+1, 1)
    test_x_list.append(tpcol)

test_x = np.array(test_x_list)
print(test_x.shape)

# Read test label
Sheet_1 = ExcelFile.sheet_by_index(3)
test_y = np.array(Sheet_1.col_values(0,1))
print(len(test_y))

#-------------------------------------------------------------------------------------------------------------    
train_x=train_x.T
test_x=test_x.T

#Take a wavelet transform
waveletname='db3'                      #commonly used wavelet functions
                                       #db1, db3, db5
                                       #coif1, coif3, coif5
                                       #bior3.1, bior3.3, bior3.5
                                       #sym2, sym4, sym6

signallen = train_x.shape[0]
samplenum = train_x.shape[1]
print("signallen："+str(signallen))
print("samplenum："+str(samplenum))

if signallen % 2 !=0:
    train_x = np.r_[train_x, np.zeros((1,samplenum))]

for i in range(samplenum):
    
    if np.mod(i,50) == 0:
        print ("Starting SWT transform: %d / %d"%(i, samplenum))
 
    data = train_x[:,i]
           
    for j in range(3):                          #The number of decomposition layers was set to 3     
        coef = pywt.swt(data, waveletname, level=1)
        data = coef[0][0]

    if i==0:
        train_x_wavelet=data
    else:
        train_x_wavelet = np.c_[train_x_wavelet, data]
#---------------------------------------------------------------------------
waveletname='db3'

signallen = test_x.shape[0]
samplenum = test_x.shape[1]
print("signallen："+str(signallen))
print("samplenum："+str(samplenum))

if signallen % 2 !=0:
    test_x = np.r_[test_x, np.zeros((1,samplenum))]

for i in range(samplenum):
    
    if np.mod(i,50) == 0:
        print ("Starting SWT transform: %d / %d"%(i, samplenum))
 
    data = test_x[:,i]
           
    for j in range(3):                              #The number of decomposition layers was set to 3     
        coef = pywt.swt(data, waveletname, level=1)
        data = coef[0][0]

    if i==0:
        test_x_wavelet=data
    else:
        test_x_wavelet = np.c_[test_x_wavelet, data]
#-------------------------------------------------------------------------------------------------------------

train_x=train_x_wavelet.T
print(train_x.shape)
test_x=test_x_wavelet.T
print(test_x.shape)
    
# Scale data
if _SCALE_FLAG_ == 1:
    normalizer = preprocessing.Normalizer().fit(train_x)
    train_x = normalizer.transform(train_x)
    test_x = normalizer.transform(test_x)

In [ ]:
Sheet_1 = ExcelFile.sheet_by_index(0)
train_x_gene = np.array(Sheet_1.col_values(0,1))
print(train_x_gene)


In [ ]:
# Set the constants
n_class = 2


_CUSTOM_FILTER_NUMBER_=[64, 64]
_CUSTOM_KERNEL_SIZE_=[4, 4]
_CUSTOM_POOL_SIZE_=[4, 4]
_CUSTOM_STRIDES_=2 
_CUSTOM_DROP_RATE_=0.5 
_CUSTOM_BATCH_SIZE_=80
_CUSTOM_EPOCHS_=200
_CUSTOM_SPLIT_RATE_=0.1
_CUSTOM_OPT_FUNCTION_='RMSprop'             # Usable functions:  SGD
                                            #                    RMSprop
                                            #                    Adagrad
                                            #                    Adadelta
                                            #                    Adam
                                            #                    Adamax
                                            #                    Nadam
_CUSTOM_LOSS_FUNCTION_='binary_crossentropy'    # Usable functions: mse / mean_squared_error
                                                #                   mae / mean_absolute_error
                                                #                   mape / mean_absolute_percentage_error
                                                #                   msle / mean_squared_logarithmic_error
                                                #                   squared_hinge
                                                #                   hinge
                                                #                   categorical_hinge
                                                #                   binary_crossentropy
                                                #                   logcosh
                                                #                   categorical_crossentropy
                                                #                   cosine_proximity
_CUSTOM_ACT_FUNCTION_='relu'     # Usable functions:    softmax
                                 #                      elu
                                 #                      selu
                                 #                      softplus
                                 #                      softsign
                                 #                      relu
                                 #                      tanh
                                 #                      sigmoid
                                 #                      hard_sigmoid
                                 #                      linear
_CUSTOM_ACT_FUNCTION_OUTPUT_LAYER_='softmax'

In [ ]:
from keras import optimizers
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os


sum_loss=0
sum_test_accuracy=0
sum_train_accuracy=0
sum_auc_train=0
sum_auc_test=0


test_accuracy=[]
train_accuracy=[]
test_auc=[]
train_auc=[]



trainning_times=100

for i in range(trainning_times):
    print("model training"+str(i+1)+":")
    
    os.mkdir('result/100times_detail/OV/'+str(i+1))
    
    
    #Reshape the training data and test data
    Mtraindata = np.expand_dims(train_x, axis = 2)
    Mtestdata = np.expand_dims(test_x, axis = 2)
    datashape = Mtraindata.shape
    featureLen = datashape[1]
    input_dim = 1
    
    # Recompile the Class Labels
    Ori_Vtrainlabel = train_y
    Ori_Vtestlabel = test_y

    Vtrainlabel = np_utils.to_categorical(train_y, n_class)
    Vtestlabel = np_utils.to_categorical(test_y, n_class)
    
    # Model construction
    model = Sequential()

    model.add(Conv1D(filters=_CUSTOM_FILTER_NUMBER_[0],
                 kernel_size=_CUSTOM_KERNEL_SIZE_[0],
                 padding='same',
                 activation=_CUSTOM_ACT_FUNCTION_,
                 strides=_CUSTOM_STRIDES_,
                 input_shape=(featureLen, input_dim)))
    model.add(MaxPooling1D(pool_size = _CUSTOM_POOL_SIZE_[0], strides=None, padding='same'))
    
    
    """ 
    model.add(Conv1D(filters=_CUSTOM_FILTER_NUMBER_[0],
                 kernel_size=_CUSTOM_KERNEL_SIZE_[0],
                 padding='same',
                 activation=_CUSTOM_ACT_FUNCTION_,
                 strides=_CUSTOM_STRIDES_)) 
    #model.add(Dropout(_CUSTOM_DROP_RATE_))
    model.add(MaxPooling1D(pool_size = _CUSTOM_POOL_SIZE_[0], strides=None, padding='same'))
    """ 
    
    #model.add(BatchNormalization())
    model.add(Flatten())
    
    
    #model.add(Dense(n_class, kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.001)))
    model.add(Dense(n_class))
    
    model.add(Dropout(0.6))
        
    model.add(Activation(_CUSTOM_ACT_FUNCTION_OUTPUT_LAYER_))

    model.compile(loss=_CUSTOM_LOSS_FUNCTION_, optimizer=_CUSTOM_OPT_FUNCTION_, metrics=['binary_accuracy'])

    #model.summary()

    hist = model.fit(Mtraindata, Vtrainlabel, 
                 batch_size =_CUSTOM_BATCH_SIZE_,
                 epochs=_CUSTOM_EPOCHS_,
                 verbose=0,
                 shuffle=True,
                 validation_split=_CUSTOM_SPLIT_RATE_)

    score = model.evaluate(Mtestdata, Vtestlabel, verbose=0)
    
    print('Test loss:', score[0])
    
    sum_loss+=score[0]
    
    
    tr_predict = model.predict(Mtraindata)
    test_predict = model.predict(Mtestdata)


    train_score_y=tr_predict[:,1]
    
    test_score_y=test_predict[:,1]
    
    
    result_df1=pd.DataFrame(train_score_y)
    result_df1.columns=['train_score_y']
    result_df1.to_csv('result/100times_detail/OV/'+str(i+1)+'/OV_100times_CNN_train_score_y.csv',index=False)
    result_df2=pd.DataFrame(test_score_y)
    result_df2.columns=['test_score_y']
    result_df2.to_csv('result/100times_detail/OV/'+str(i+1)+'/OV_100times_CNN_test_score_y.csv',index=False)
    
    
    fpr1, tpr1, thresholds1 = roc_curve(train_y.astype(int), train_score_y, pos_label=1)
    auc1= auc(fpr1,tpr1)
    sum_auc_train+=auc1
    train_auc.append(auc1)
    print("AUC Score (Training):  %5.4f"%auc1)
    fpr2, tpr2, thresholds2 = roc_curve(test_y.astype(int), test_score_y, pos_label=1)
    auc2= auc(fpr2,tpr2)
    sum_auc_test+=auc2
    test_auc.append(auc2)
    print("AUC Score (Test):  %5.4f"%auc2)
    
    
    tr_predict = tr_predict.argmax(1)
    test_predict = test_predict.argmax(1)
    
    result_df3=pd.DataFrame(tr_predict)
    result_df3.columns=['tr_predict']
    result_df3.to_csv('result/100times_detail/OV/'+str(i+1)+'/OV_100times_CNN_tr_predict.csv',index=False)
    result_df4=pd.DataFrame(test_predict)
    result_df4.columns=['test_predict']
    result_df4.to_csv('result/100times_detail/OV/'+str(i+1)+'/OV_100times_CNN_test_predict.csv',index=False)
    
    acc_train=metrics.accuracy_score(Ori_Vtrainlabel.astype(int), tr_predict)
    sum_train_accuracy+=acc_train
    train_accuracy.append(acc_train)
    print("ACC Score (Training):  %5.4f"%acc_train)
    
    acc_test=metrics.accuracy_score(Ori_Vtestlabel.astype(int), test_predict)
    sum_test_accuracy+=acc_test
    test_accuracy.append(acc_test)
    print("ACC Score (Test):  %5.4f"%acc_test)
    
    model.save('result/100times_detail/OV/'+str(i+1)+'/OV_model.h5')
    
    
    if(i==0):
        BEST_MODEL=model
        BEST_AUC=auc2
        model.save("result/CNN_model/OV_SWT_CNN_best.h5")
        print('The best AUC is:'+str(BEST_AUC))
    if(auc2>BEST_AUC):
        BEST_MODEL=model
        BEST_AUC=auc2
        model.save("result/CNN_model/OV_SWT_CNN_best.h5")
        print('The best AUC is:'+str(BEST_AUC))
    
  
    
print('Average train accuracy:', sum_train_accuracy/trainning_times)
print('Average test accuracy:', sum_test_accuracy/trainning_times)
print('Average train auc:', sum_auc_train/trainning_times)
print('Average test auc:', sum_auc_test/trainning_times)
print('The best AUC is:'+str(BEST_AUC))


accuracy_df1=pd.DataFrame(train_accuracy)
accuracy_df1.columns=['train_accuracy']
accuracy_df1.to_csv('result/100times/OV/OV_100times_CNN_train_accuracy.csv',index=False)
accuracy_df2=pd.DataFrame(test_accuracy)
accuracy_df2.columns=['test_accuracy']
accuracy_df2.to_csv('result/100times/OV/OV_100times_CNN_test_accuracy.csv',index=False)

accuracy_df3=pd.DataFrame(train_auc)
accuracy_df3.columns=['train_auc']
accuracy_df3.to_csv('result/100times/OV/OV_100times_CNN_train_auc.csv',index=False)
accuracy_df4=pd.DataFrame(test_auc)
accuracy_df4.columns=['test_auc']
accuracy_df4.to_csv('result/100times/OV/OV_100times_CNN_test_auc.csv',index=False)

In [ ]:
#load the model
from keras.models import load_model

test_model=load_model("data/OV_SWT_CNN_best.h5")             #Change to the path that save the optimal model

tr_predict = test_model.predict(Mtraindata)
test_predict = test_model.predict(Mtestdata)

train_score_y=tr_predict[:,1]
test_score_y=test_predict[:,1]
    
fpr1, tpr1, thresholds1 = roc_curve(train_y.astype(int), train_score_y, pos_label=1)
auc1= auc(fpr1,tpr1)
print("AUC Score (Training):  %5.4f"%auc1)
fpr2, tpr2, thresholds2 = roc_curve(test_y.astype(int), test_score_y, pos_label=1)
auc2= auc(fpr2,tpr2)
print("AUC Score (Test):  %5.4f"%auc2)
    
    
tr_predict = tr_predict.argmax(1)
test_predict = test_predict.argmax(1)
    
acc_train=metrics.accuracy_score(Ori_Vtrainlabel.astype(int), tr_predict)
print("ACC Score (Training):  %5.4f"%acc_train)
    
acc_test=metrics.accuracy_score(Ori_Vtestlabel.astype(int), test_predict)
print("ACC Score (Test):  %5.4f"%acc_test)

In [ ]:
test_model.summary()

In [ ]:
print(test_model.layers[0].input)
print(test_model.layers[0].output)

print(test_model.layers[1].output)

print(test_model.layers[5].output)

In [ ]:
from keras import backend as K

#-------------------------------pool------------------------------------------
get_layer1_output = K.function([test_model.layers[0].input],[test_model.layers[1].output])
layer1_output = get_layer1_output([Mtraindata])[0]
print(layer1_output.shape)

for i in range(layer1_output.shape[0]):
    a=layer1_output[i].mean(axis=1)    
    if i==0:
        layer1_matrix=a
    else:
        layer1_matrix = np.c_[layer1_matrix, a]
        
print(layer1_matrix.shape)

train_x=train_x.T
print(train_x.shape)

In [ ]:
#--------------------------calculate B matrix----------------------------------
temp_a=np.dot(layer1_matrix,layer1_matrix.T)
print(temp_a)
temp_b=np.linalg.inv(temp_a)
print(temp_b)

temp_c=np.dot(train_x,layer1_matrix.T)
B=np.dot(temp_c,temp_b)
print(B.shape)

score=(B.sum(1))/B.shape[1]
print(score)
#score=np.delete(score,len(score)-1,0)     #Execute this command if the original signal length is odd
print(score)
print(score.shape)
output_df=pd.DataFrame(score,index=train_x_gene)
output_df.columns=['score']
output_df.to_csv('result/OV/OV_pool_mean_score.csv',index="gene")

scores=pd.DataFrame(score,columns=['score'],index=train_x_gene)
geneSorted=scores.sort_values(by=['score'],ascending=False)
print(geneSorted)

In [ ]:
train_x = pd.read_excel('data/Dataset_OV.xlsx','train_x',index_col="gene")

temp_output_topGene=[]

geneList=geneSorted.index
print(geneList)

N=1000
for i in range(N):
    if(geneList[i] not in train_x.index):
        temp_output_topGene.append(np.zeros((1,len(train_x.columns))))
        continue
    temp=train_x.loc[geneList[i]]
    temp_output_topGene.append(np.array(temp))

output_topGene=pd.DataFrame(temp_output_topGene,columns=train_x.columns,index=geneList[0:1000:1])


output_topGene=output_topGene.T
print(output_topGene)
output_topGene.to_csv('result/OV/OV_top1000Gene.csv',index="id")

In [ ]:
#Calculate the scores of each sample after the cox regression
import pandas as pd
import numpy as np

cox_result=pd.read_csv('data/OV_clinical_MultipleResult_0.05.csv',sep=',',index_col="Characteristics")   #the gene list after the Multivariable Cox regression(p < 0.05) 
print(cox_result)
cox_gene=cox_result.index
print(cox_gene)


train_x = pd.read_excel('data/Dataset_OV.xlsx','train_x',index_col="gene")
test_x = pd.read_excel('data/Dataset_OV.xlsx','test_x',index_col="gene")
train_x_sample=train_x.columns 
test_x_sample=test_x.columns

train_sample_score=np.zeros((1,len(train_x_sample)))
test_sample_score=np.zeros((1,len(test_x_sample))) 

for i in range(len(cox_gene)):
    temp=train_x.loc[cox_gene[i]]
    train_sample_score+=np.array(temp)*cox_result.loc[cox_gene[i]]['Coefficient']
    
for i in range(len(cox_gene)):
    temp=test_x.loc[cox_gene[i]]
    test_sample_score+=np.array(temp)*cox_result.loc[cox_gene[i]]['Coefficient']

train_sample_score=train_sample_score.T
print(train_sample_score.shape)
test_sample_score=test_sample_score.T
print(test_sample_score.shape)
    

train_sample_score=pd.DataFrame(train_sample_score,index=train_x_sample)
test_sample_score=pd.DataFrame(test_sample_score,index=test_x_sample)

    
writer=pd.ExcelWriter('result/OV_cox0.05_Sample_score.xlsx')


train_sample_score.to_excel(writer,sheet_name='train_x',index_label="sample")
test_sample_score.to_excel(writer,sheet_name='test_x',index_label="sample")

writer.save()  
